In [1]:
import json 
from collections import Counter
from tqdm import tqdm 
from colorama import Style,Fore

## Refining 1

- Refine the language field of the videos

In [10]:
from utils import RefineLanguage1


In [10]:
RefineLanguage1()

refining the language...: 100%|██████████| 6859/6859 [01:53<00:00, 60.57it/s]



 json saved


In [11]:
with open("../jsons/videosR1.json", "r", encoding="utf-8") as file:
    videosR1 = json.load(file)
len(videosR1)

6859

In [29]:
langue_counter = Counter(video['langue'] for video in videosR1)
print("Toutes les valeurs uniques de langue avec leur nombre d'occurrences :")
for langue, count in langue_counter.items():
    print(f"- {langue} : {count}")


Toutes les valeurs uniques de langue avec leur nombre d'occurrences :
- fr : 6276
- autre : 583


## Refining 2

- Add the Localisation Information

In [18]:
from utils import RefineLanguage2

In [19]:
RefineLanguage2()

Refining language...: 100%|██████████| 6859/6859 [00:00<00:00, 340575.24it/s]



 json saved


In [5]:
with open("../jsons/videosR2.json", "r", encoding="utf-8") as file:
    videosR2 = json.load(file)
len(videosR2)

6859

In [25]:
langue_counter = Counter(video['langue'] for video in videosR2)
print("Toutes les valeurs uniques de langue avec leur nombre d'occurrences :")
for langue, count in sorted(langue_counter.items(), key=lambda x: x[1], reverse=True):
    print(f"- {langue} : {count}")


Toutes les valeurs uniques de langue avec leur nombre d'occurrences :
- fr-FR : 4312
- fr : 1332
- autre : 583
- fr-CA : 229
- fr-BE : 112
- fr-CH : 88
- fr-US : 49
- fr-MA : 35
- fr-NC : 14
- fr-PT : 14
- fr-DE : 11
- fr-IT : 10
- fr-RE : 8
- fr-ES : 7
- fr-SN : 6
- fr-MQ : 6
- fr-DZ : 5
- fr-CI : 4
- fr-GB : 4
- fr-AR : 2
- fr-NL : 2
- fr-CM : 2
- fr-ZA : 2
- fr-PF : 2
- fr-TH : 2
- fr-GP : 2
- fr-AE : 2
- fr-TR : 1
- fr-TG : 1
- fr-HK : 1
- fr-MU : 1
- fr-RU : 1
- fr-IN : 1
- fr-SA : 1
- fr-KE : 1
- fr-KR : 1
- fr-LU : 1
- fr-IE : 1
- fr-LK : 1
- fr-BJ : 1
- fr-TN : 1


In [6]:
total = len(videosR2)

count_fr = sum(1 for video in videosR2 if video['langue'] == 'fr')
count_frFR = sum(1 for video in videosR2 if video['langue'] == 'fr-FR')
count_fr_autre = sum(1 for video in videosR2 if video['langue'].startswith('fr') and video['langue'] not in ['fr', 'fr-FR'])
count_autre = total - (count_fr + count_frFR + count_fr_autre)

print(f"Total de vidéos : {total}")
print(f"- fr       : {count_fr} ({count_fr / total:.2%})")
print(f"- fr-FR    : {count_frFR} ({count_frFR / total:.2%})")
print(f"- fr-autre : {count_fr_autre} ({count_fr_autre / total:.2%})")
print(f"- autre    : {count_autre} ({count_autre / total:.2%})")

Total de vidéos : 6859
- fr       : 1332 (19.42%)
- fr-FR    : 4312 (62.87%)
- fr-autre : 632 (9.21%)
- autre    : 583 (8.50%)


In [30]:
1332+4312+632

6276

## Refining 3

- Get the localisation information for 'fr' videos :
  - Try to explore the video description+Title and Bio of the corresponding channel.

### 1.Seperate These videos 

In [13]:
with open("../jsons/videosR2.json", "r", encoding="utf-8") as file:
    videosR2 = json.load(file)
videosfr = []
for video in videosR2:
    if video['langue']=='fr':
        videosfr.append(video)
with open("../jsons/videosfr.json", "w", encoding="utf-8") as f:
    json.dump(videosfr, f, ensure_ascii=False, indent=2)
    print(Style.BRIGHT+Fore.GREEN+'\n json saved'+Style.RESET_ALL)
    print(Style.BRIGHT+Fore.YELLOW+f'\n json length is : {len(videosfr)}'+Style.RESET_ALL)


 json saved

 json length is : 1332


### 2. Get the location by using metadata:

In [15]:
# Get the Context first
import importlib
import utils  # or whatever your module is called
importlib.reload(utils)

from utils import getContextAll


In [16]:
getContextAll()

100%|██████████| 1332/1332 [00:04<00:00, 277.08it/s]



 json saved

 json length is : 1332


In [17]:
st = "Le bio de la chaine : \n \n        \n###############\n        \nDescription : Tout savoir sur le torchis et son application dans les maisons traditionnelles des Vosges du Nord !\n\nJacques Bruderer, Maître maçon et tailleur de pierre, artisan spécialiste du bâti ancien, répond à toutes les questions qu’on peut se poser avant de se lancer. \n- Quel type de terre est utilisée dans notre Région ? Est-ce qu’elle peut comporter des gravats ?\n- Est-ce qu’on peut utiliser la terre de son propre terrain ? \n- Quelle paille utilisée ?\n- Le torchis est-il porteur ?\n- Comment préparer le mélange ? Quelle doit-être la texture ?\n- Peut-on réemployer un ancien torchis ?\n\nVous trouverez toutes les informations sur les caractéristiques des matériaux terre et paille, la préparation du matériel et des mélanges puis la technique de mise en œuvre.  \n\nLien vers la réglementation : \nGuide des bonnes pratiques\nDe la construction en terre crue\nEdition du 13 décembre 2018\nhttps://www.ekopolis.fr/sites/default/files/2019-04/RES-1904-GBP-TerreCrue-COMPLET-V01_web.pdf\n\nOù se fournir en terre locale ? http://www.brique-lanter.com/\n\nEt pour aller plus loin, visitez notre site internet : https://eco-renover.parc-vosges-nord.fr/\n\nChapitres :\n00:00 - Introduction\n00:30 - Présentation générale\n03:19 - Pourquoi le torchis?\n04:17 - Chantier du jour\n06:06 - Système constructif d'un miroir\n10:46 - Système constructif d'un plafond\n13:35 - Réparer un plafond\n16:26 - Composition du torchis\n20:41 - Le torchis bien fait\n23:29 - Préparer le torchis\n24:16 - Appliquer le torchis\n28:53 - Ce qu'il faut savoir sur la pose du torchis\n31:26 - Finition du torchis\n34:21 - Le torchis du plafond\n39:20 - Les avantages du torchis\n40:49 - Résumé\n42:24 - Outro\n\n\n\nTags :\nparc, naturel, régional, vosges, nord, pnr, vosges-du-nord, torchis, torchis colombage, paille et argile, torchis maison, torchis plafond, torchis mur, rénovation naturelle, rénovation maison, rénovation traditionnelle, maison traditionnelle, éco rénovation\n        \n        \n###############\n        \nDescription : Une série de portraits de producteurs dans le Parc naturel régional des Vosges du nord\n        \n        \n###############\n\n        titre : Comment réaliser du torchis dans un colombage ?\n        \n        \n###############\n\n        titre : Portrait de Claude Ramspacher - ferme du Herrenstein, éleveur fromager à Neuwiller-lès-Saverne (67)\n        "
print(st)

Le bio de la chaine : 
 
        
###############
        
Description : Tout savoir sur le torchis et son application dans les maisons traditionnelles des Vosges du Nord !

Jacques Bruderer, Maître maçon et tailleur de pierre, artisan spécialiste du bâti ancien, répond à toutes les questions qu’on peut se poser avant de se lancer. 
- Quel type de terre est utilisée dans notre Région ? Est-ce qu’elle peut comporter des gravats ?
- Est-ce qu’on peut utiliser la terre de son propre terrain ? 
- Quelle paille utilisée ?
- Le torchis est-il porteur ?
- Comment préparer le mélange ? Quelle doit-être la texture ?
- Peut-on réemployer un ancien torchis ?

Vous trouverez toutes les informations sur les caractéristiques des matériaux terre et paille, la préparation du matériel et des mélanges puis la technique de mise en œuvre.  

Lien vers la réglementation : 
Guide des bonnes pratiques
De la construction en terre crue
Edition du 13 décembre 2018
https://www.ekopolis.fr/sites/default/files/201

## Filtering 1

- Remove fr-autre,autre
- keep only fr and fr-FR , a big amount of fr are for france.
- obtain the localisation of a channel if it's not providled by youtube api can be complexe because you can't find it anywhere.

In [4]:
with open("../jsons/videosR2.json", "r", encoding="utf-8") as file:
    videosR2 = json.load(file)
len(videosR2)

6859